In [1]:
import os
import pandas as pd
import json
import time
from datetime import datetime, date
import requests

# from bs4 import BeautifulSoup
# import math
# import httpx
# import asyncio
# import nest_asyncio
from urllib.request import urlopen

# import ftfy

os.chdir('C:/Users/Gwool/Documents/Python Scripts/Pricing/Exports/Consum')

Functions

In [2]:
def get_price_consum():
    url = "https://tienda.consum.es/api/rest/V1.0/shopping/category/menu"
    data_json_cat = requests.get(url=url).json()

    categories_desc = []

    for j in range(len(data_json_cat)):
        if data_json_cat[j]['id'] == 99999:
            continue
        else:
            cat_id = data_json_cat[j]['id']
            cat_name = data_json_cat[j]['nombre']

            offset = 0
            limit = 100
            has_more_results = True

            while has_more_results:
                url = f"https://tienda.consum.es/api/rest/V1.0/catalog/product?limit={limit}&offset={offset}&orderById=5&showRecommendations=false&categories={cat_id}"

                data_json = requests.get(url = url).json()

                for i in range(len(data_json['products'])):
                    try:
                        prod_id   = data_json['products'][i]['id']
                        prod_code = data_json['products'][i]['code']
                        prod_ean  = data_json['products'][i]['ean']

                        # Prod data
                        prod_name       = data_json['products'][i].get('productData').get('name')
                        prod_desc       = data_json['products'][i].get('productData').get('description')
                        prod_brand_id   = data_json['products'][i].get('productData').get('brand').get('id')
                        prod_brand_name = data_json['products'][i].get('productData').get('brand').get('name')

                        # Price values - Raw and disc

                        if len(data_json['products'][0]['priceData'].get('prices')) > 1:
                            prod_unit     = data_json['products'][i].get('priceData').get('unitPriceUnitType')
                            prod_min_unit = data_json['products'][i].get('priceData').get('minimumUnit')

                            prod_price_id_raw   = data_json['products'][i]['priceData'].get('prices')[0].get('id')
                            prod_price_val_raw  = data_json['products'][i]['priceData'].get('prices')[0].get('value').get('centAmount')
                            prod_price_unit_raw = data_json['products'][i]['priceData'].get('prices')[0].get('value').get('centUnitAmount')

                            prod_price_id_disc   = data_json['products'][i]['priceData'].get('prices')[1].get('id')
                            prod_price_val_disc  = data_json['products'][i]['priceData'].get('prices')[1].get('value').get('centAmount')
                            prod_price_unit_disc = data_json['products'][i]['priceData'].get('prices')[1].get('value').get('centUnitAmount')

                        else:
                            prod_unit     = data_json['products'][i].get('priceData').get('unitPriceUnitType')
                            prod_min_unit = data_json['products'][i].get('priceData').get('minimumUnit')

                            prod_price_id_raw   = data_json['products'][i]['priceData'].get('prices')[0].get('id')
                            prod_price_val_raw  = data_json['products'][i]['priceData'].get('prices')[0].get('value').get('centAmount')
                            prod_price_unit_raw = data_json['products'][i]['priceData'].get('prices')[0].get('value').get('centUnitAmount')

                            prod_price_id_disc   = 'OFFER_PRICE' 
                            prod_price_val_disc  = prod_price_val_raw 
                            prod_price_unit_disc = prod_price_unit_raw 
                        
                        categories_desc.append([cat_id, cat_name, prod_id, prod_code, prod_ean, prod_name, prod_desc, prod_brand_id, prod_brand_name, prod_unit, prod_min_unit, prod_price_id_raw, prod_price_val_raw, prod_price_unit_raw, prod_price_id_disc, prod_price_val_disc, prod_price_unit_disc])

                    except:
                        pass

                total_count = data_json['totalCount']
                current_count = offset + limit
                has_more_results = current_count < total_count

                # Actualizar el offset para la siguiente página
                offset += limit

                # print(prod_desc, prod_brand_name)

    return pd.DataFrame(categories_desc, columns=['cat_id', 'cat_name', 'prod_id', 'prod_code', 'prod_ean', 'prod_name', 'prod_desc', 'prod_brand_id', 'prod_brand_name', 'prod_unit', 'prod_min_unit', 'prod_price_id_raw', 'prod_price_val_raw', 'prod_price_unit_raw', 'prod_price_id_disc', 'prod_price_val_disc', 'prod_price_unit_disc'])


Run and export

In [3]:
#Create a iteration date folder to save the results
date = datetime.now().strftime('%Y_%m_%d')
folder = os.path.join(r'C:\Users\Gwool\Documents\Python Scripts\Pricing\Exports\Consum', date)
if not os.path.isdir(folder):
    print(f'Need to Create Folder')
    os.mkdir(folder)
if os.path.isdir(folder):
    print('Folder Created')

Need to Create Folder
Folder Created


In [4]:
Consum_prices = get_price_consum()
Consum_prices['Period'] = str(datetime.today().year) + str(datetime.today().month).zfill(2)
Consum_prices['Year'  ] = datetime.today().year
Consum_prices['Month' ] = datetime.today().month

export = os.path.join(folder, 'Export_Pricing_Total_Consum.xlsx')
Consum_prices.to_excel(export, index = False)
print(Consum_prices.shape[0], 'Products saved correctly')

8539 Products saved correctly


Total Execution

In [5]:
total = pd.read_csv('./Total_Prices/Total_Consum_Prices.csv')

if datetime.now().month not in total.Month.unique():
    total = pd.concat([total, Consum_prices], ignore_index=True)
    total.drop_duplicates(keep='first', inplace = True)
    total.to_csv('./Total_Prices/Total_Consum_Prices.csv', encoding='utf-8-sig', index=False)
    total['period'] = total.Year.astype(str) + total.Month.astype(str).str.zfill(2)

    print('Total FIle exported corretly.')
else:
    print(f'Month {datetime.now().month} already in Total file')

Total FIle exported corretly.


End